In [1]:
from scripts.data_prep import process_event_batches
from scripts.reconstruct import infer_graph, get_tracks
from scripts.visualize import plot_tracks_compare, stats
import scripts.utils as utils
from scripts.gnn_model import gnn
import torch as torch
import numpy as np

# Inferencia y reconstrucción individual

In [ ]:
n_event ='000001000'
# OJO: utilizar utilizar el modelo entrenado con el filtro que se vaya a usar (si filtro: real ==> modelo: real)
#carga de modelo y sus hiperparámetros (todos han sido entrenados con config2, archivo de hiperparámetros)
model_hyper, preprocess_hyper = utils.load_hyper(name = 'config2')


#condensamos toda la reconstrucción en una sola función
def reconstruct_event(n_event, ideal): 
    trained_model = gnn(**model_hyper)
    if ideal:
        pt_min = 0.5
        ruta_modelo = 'models/ideal/model_f1_0.9452604_config_2.pt'
    else:
        pt_min = 0 #permite que actúe el filtro real
        ruta_modelo = 'models/real/model_f1_0.8001674_config_2_pt_filter.pt'

    trained_model.load_state_dict(torch.load(ruta_modelo))
    trained_model.eval()

    prepared_graphs, ground_tracks = process_event_batches(n_event, drop_y=True, get_true_tracks = True, pt_min=pt_min, pt_max=1000, dtheta=np.pi*2, dphi=np.pi) #creación de grafos

    tracks_batches = []
    for i in prepared_graphs:
        predicted_graph = infer_graph(trained_model, i, threshold=False) #inferencia, probabilidades sin filtro de umbral
        df_tracks = get_tracks(predicted_graph) #algoritmo CTD
        if df_tracks is not None:
            tracks_batches.append(df_tracks)
    pred_tracks = utils.concatenar_track_ids(tracks_batches)
    purity, efficency = stats(pred_tracks, ground_tracks, n_matched_hits=5)
    pred_tracks, ground_tracks = utils.align_track_id(pred_tracks, ground_tracks) #el mismo track en ambos conjuntos tiene el mismo track_id

    return pred_tracks, ground_tracks, purity, efficency
    
pred_tracks, ground_tracks, pureza, eficiencia = reconstruct_event(n_event, ideal = True)

Tracks descartados por no empezar en capas 0, 1 o 2: 58
Tracks descartados por capas no consecutivas: 121
Tracks descartados por no tener entre 7 y 10 hits: 446
Reconstrucción de tracks con minimo 7 hits y matches de 5 o más
Purity: 98.48% (1038/1054)
Efficiency: 75.22% (1038/1380)


In [8]:
plot_tracks_compare(ground_tracks, pred_tracks)

# Eficiencia y purezas medias con filtro real

In [ ]:
#carga de modelo y sus hiperparámetros (todos han sido entrenados con config2, archivo de hiperparametros)
model_hyper, preprocess_hyper = utils.load_hyper(name = 'config2')
trained_model = gnn(**model_hyper)

# MODELO entrenado con dataset real
ruta_modelo = 'models/real/model_f1_0.8001674_config_2_pt_filter.pt'
trained_model.load_state_dict(torch.load(ruta_modelo))
trained_model.eval()

# Número de eventos
num_events = 10
start_event = 1000 

eficiencias = []
purezas = []

for i in range(num_events):
    n_event = f'{start_event + i:09d}'  
    try:
        pred_tracks, ground_tracks, pureza, eficiencia = reconstruct_event(n_event, trained_model, ideal=False)
        purezas.append(pureza)
        eficiencias.append(eficiencia)
    except Exception as e:
        print(f"Error en el evento {n_event}: {e}")

eficiencias = np.array(eficiencias)
purezas = np.array(purezas)

media_eficiencia = np.mean(eficiencias)
std_eficiencia = np.std(eficiencias)
media_pureza = np.mean(purezas)
std_pureza = np.std(purezas)

print(f"Eficiencia: media = {media_eficiencia:.4f}, desviación = {std_eficiencia:.4f}")
print(f"Pureza:     media = {media_pureza:.4f}, desviación = {std_pureza:.4f}")


Tracks descartados por no empezar en capas 0, 1 o 2: 58
Tracks descartados por capas no consecutivas: 121
Tracks descartados por no tener entre 7 y 10 hits: 446
Reconstrucción de tracks con minimo 7 hits y matches de 5 o más
Purity: 98.55% (613/622)
Efficiency: 44.42% (613/1380)
Tracks descartados por no empezar en capas 0, 1 o 2: 34
Tracks descartados por capas no consecutivas: 67
Tracks descartados por no tener entre 7 y 10 hits: 258
Reconstrucción de tracks con minimo 7 hits y matches de 5 o más
Purity: 97.17% (481/495)
Efficiency: 49.33% (482/977)
Tracks descartados por no empezar en capas 0, 1 o 2: 64
Tracks descartados por capas no consecutivas: 98
Tracks descartados por no tener entre 7 y 10 hits: 426
Reconstrucción de tracks con minimo 7 hits y matches de 5 o más
Purity: 97.66% (667/683)
Efficiency: 44.08% (667/1513)
Tracks descartados por no empezar en capas 0, 1 o 2: 36
Tracks descartados por capas no consecutivas: 81
Tracks descartados por no tener entre 7 y 10 hits: 329
Rec

# Eficiencia y purezas medias con filtro ideal

In [ ]:
#carga de modelo y sus hiperparámetros (todos han sido entrenados con config2, archivo de hiperparametros)
model_hyper, preprocess_hyper = utils.load_hyper(name = 'config2')
trained_model = gnn(**model_hyper)

# MODELO entrenado con dataset real
ruta_modelo = 'models/ideal/model_f1_0.9452604_config_2.pt'
trained_model.load_state_dict(torch.load(ruta_modelo))
trained_model.eval()
 
# Número de eventos
num_events = 10
start_event = 1000 

eficiencias = []
purezas = []

for i in range(num_events):
    n_event = f'{start_event + i:09d}'  
    try:
        pred_tracks, ground_tracks, pureza, eficiencia = reconstruct_event(n_event, trained_model, ideal=True)
        purezas.append(pureza)
        eficiencias.append(eficiencia)
    except Exception as e:
        print(f"Error en el evento {n_event}: {e}")

eficiencias = np.array(eficiencias)
purezas = np.array(purezas)

media_eficiencia = np.mean(eficiencias)
std_eficiencia = np.std(eficiencias)
media_pureza = np.mean(purezas)
std_pureza = np.std(purezas)

print(f"Eficiencia: media = {media_eficiencia:.4f}, desviación = {std_eficiencia:.4f}")
print(f"Pureza:     media = {media_pureza:.4f}, desviación = {std_pureza:.4f}")

Tracks descartados por no empezar en capas 0, 1 o 2: 58
Tracks descartados por capas no consecutivas: 121
Tracks descartados por no tener entre 7 y 10 hits: 446
Reconstrucción de tracks con minimo 7 hits y matches de 5 o más
Purity: 98.48% (1038/1054)
Efficiency: 75.22% (1038/1380)
Tracks descartados por no empezar en capas 0, 1 o 2: 34
Tracks descartados por capas no consecutivas: 67
Tracks descartados por no tener entre 7 y 10 hits: 258
Reconstrucción de tracks con minimo 7 hits y matches de 5 o más
Purity: 97.47% (771/791)
Efficiency: 78.92% (771/977)
Tracks descartados por no empezar en capas 0, 1 o 2: 64
Tracks descartados por capas no consecutivas: 98
Tracks descartados por no tener entre 7 y 10 hits: 426
Reconstrucción de tracks con minimo 7 hits y matches de 5 o más
Purity: 97.66% (1127/1154)
Efficiency: 74.49% (1127/1513)
Tracks descartados por no empezar en capas 0, 1 o 2: 36
Tracks descartados por capas no consecutivas: 81
Tracks descartados por no tener entre 7 y 10 hits: 3